In [1]:
import os
import numpy as np # type: ignore
from scipy.io import loadmat # type: ignore

Librerias para trabajar con la red neuronal y procesamiento de datos

In [2]:
import tensorflow as tf # Para red neuronal profunda
import numpy as np
import matplotlib.pyplot as plt
import time # Para tomar el tiempo de entrenamiento de la red

In [3]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

##################################################################################<br>
##################### Convertir matrices de .mat a .npy ##########################<br>
##################################################################################

Directorios de entrada y salida

In [4]:
input_pam_dir = 'D:/TT/Memoria/waveletycnn/codigo_matlab/codigo_fuente/matrices_complejas_pam_mat' # matrices en formato .mat
input_vsc_dir = 'D:/TT/Memoria/waveletycnn/codigo_matlab/codigo_fuente/matrices_complejas_vsc_mat' # matrices en formato .mat
output_pam_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_pam_python' # INPUT PARA LA RED
output_vsc_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_vsc_python' # OUTPUT O SALIDAS ESPERADAS PARA LA RED

Crear los directorios de salida si no existen

In [5]:
os.makedirs(output_pam_dir, exist_ok=True)
os.makedirs(output_vsc_dir, exist_ok=True)

Funcion para convertir archivos .mat a .npy

In [6]:
def convert_mat_to_npy(input_dir, output_dir, prefix):
    for i in range(1, 31):
        mat_file = os.path.join(input_dir, f'{prefix}_noise_{i}.mat')
        npy_file = os.path.join(output_dir, f'{prefix}_noise_{i}.npy')
        
        # Cargar el archivo .mat
        mat_data = loadmat(mat_file)
        
        # Extraer la matriz compleja
        matrix_key = [key for key in mat_data.keys() if not key.startswith('__')][0]
        matrix = mat_data[matrix_key]
        
        # Guardar la matriz en formato .npy
        np.save(npy_file, matrix)

Convertir archivos .mat a .npy para PAM y VSC

In [7]:
convert_mat_to_npy(input_pam_dir, output_pam_dir, 'matrix_complex_pam')
convert_mat_to_npy(input_vsc_dir, output_vsc_dir, 'matrix_complex_vsc')

###################################################################################<br>
##################### Conversion a tensor tridimensional ##########################<br>
###################################################################################

Directorios de entrada y salida

In [8]:
input_pam_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_pam_python'
input_vsc_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_vsc_python'
output_pam_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_pam_procesadas'
output_vsc_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_vsc_procesadas'

Crear directorios de salida si no existen

In [9]:
os.makedirs(output_pam_dir, exist_ok=True)
os.makedirs(output_vsc_dir, exist_ok=True)

FunciÃ³n para procesar las matrices complejas

In [10]:
def procesar_matriz_compleja(matriz_compleja):
    datos_organizados = np.stack((matriz_compleja.real, matriz_compleja.imag), axis=-1)
    return datos_organizados

Procesar matrices complejas en la carpeta input_pam_dir

In [11]:
for filename in os.listdir(input_pam_dir):
    if filename.endswith('.npy'):
        input_path = os.path.join(input_pam_dir, filename)
        output_path = os.path.join(output_pam_dir, filename)
        
        # Cargar la matriz compleja
        matriz_compleja = np.load(input_path)
        
        # Procesar la matriz compleja
        datos_organizados = procesar_matriz_compleja(matriz_compleja)
        
        # Guardar los datos procesados
        np.save(output_path, datos_organizados)

Procesar matrices complejas en la carpeta input_vsc_dir

In [12]:
for filename in os.listdir(input_vsc_dir):
    if filename.endswith('.npy'):
        input_path = os.path.join(input_vsc_dir, filename)
        output_path = os.path.join(output_vsc_dir, filename)
        
        # Cargar la matriz compleja
        matriz_compleja = np.load(input_path)
        
        # Procesar la matriz compleja
        datos_organizados = procesar_matriz_compleja(matriz_compleja)
        
        # Guardar los datos procesados
        np.save(output_path, datos_organizados)

In [13]:
print("Procesamiento completado.")

Procesamiento completado.


###################################################################################<br>
################## Verificacicon de "shape" - matrices pam y vsc ##################<br>
###################################################################################

Directorios de salida

In [14]:
output_pam_dir_check = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_pam_procesadas'
output_vsc_dir_check = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_vsc_procesadas'

Funcion para verificar la forma de una matriz

In [15]:
def verificar_shape(directorio, nombre_archivo):
    path = os.path.join(directorio, nombre_archivo)
    matriz = np.load(path)
    return matriz.shape

Verificar la forma de un archivo de ejemplo en output_pam_dir_check

In [16]:
ejemplo_pam = os.listdir(output_pam_dir_check)[0]  # Obtener el primer archivo de la carpeta
shape_pam = verificar_shape(output_pam_dir_check, ejemplo_pam)
print(f"Shape de {ejemplo_pam} en {output_pam_dir_check}: {shape_pam}")

Shape de matrix_complex_pam_noise_1.npy en D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_pam_procesadas: (36, 1024, 2)


Verificar la forma de un archivo de ejemplo en output_vsc_dir_check

In [17]:
ejemplo_vsc = os.listdir(output_vsc_dir_check)[0]  # Obtener el primer archivo de la carpeta
shape_vsc = verificar_shape(output_vsc_dir_check, ejemplo_vsc)
print(f"Shape de {ejemplo_vsc} en {output_vsc_dir_check}: {shape_vsc}")

Shape de matrix_complex_vsc_noise_1.npy en D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_vsc_procesadas: (36, 1024, 2)


##################################################################################<br>
####################### Red Neuronal Profunda: U-net #############################<br>
##################################################################################

Directorios de entrada

In [18]:
input_pam_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_pam_procesadas'
output_vsc_dir = 'D:/TT/Memoria/waveletycnn/codigo_python/matrices_complejas_vsc_procesadas'

FunciÃ³n para cargar los archivos .npy

In [19]:
def load_npy_files(input_dir):
    files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.npy')])
    data = [np.load(f) for f in files]
    return np.array(data)

Cargar los datos de entrada y salida

In [20]:
X = load_npy_files(input_pam_dir) # inputs
Y = load_npy_files(output_vsc_dir) # outputs

Verificar las formas de los datos cargados

In [21]:
print(f"Shape de los inputs (X): {X.shape}")
print(f"Shape de los outputs (Y): {Y.shape}")

Shape de los inputs (X): (30, 36, 1024, 2)
Shape de los outputs (Y): (30, 36, 1024, 2)


Definir la U-Net con regularizaciÃ³n L2

In [22]:
def unet_model_with_l2(input_shape, l2_lambda):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Regularizer
    l2_reg = tf.keras.regularizers.l2(l2_lambda)
    
    # Encoder
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    
    # Bottleneck
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(p2)
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(c3)
    
    # Decoder
    u4 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u4 = tf.keras.layers.concatenate([u4, c2])
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(u4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(c4)
    
    u5 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = tf.keras.layers.concatenate([u5, c1])
    c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(u5)
    c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2_reg)(c5)
    
    outputs = tf.keras.layers.Conv2D(2, (1, 1), activation='linear')(c5)
    
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    
    return model

Definir la mÃ©trica NMSE ajustada para utilizar la varianza de los valores verdaderos

In [23]:
def nmse(y_true, y_pred):
    mse = tf.keras.backend.mean(tf.keras.backend.square(y_true - y_pred))
    var_true = tf.keras.backend.var(y_true)
    return mse / var_true

Hiperparametros

In [24]:
max_epoch = 200
batchsize = 16
learning_rate = 0.001
l2_lambda = 0.01
validation_split = 0.3 # 70% entrenamiento & 30% validacion

Definir el modelo

In [25]:
input_shape = X.shape[1:]  # forma del input a entrar. en este caso esta forma debe coincidir con las matrices que entran a la red
model = unet_model_with_l2(input_shape, l2_lambda)

DEFINICION DE LA FUNCION DE DECAIMIENTO, ALGORITMO OPTIMIZADOR, FUNCION DE PERDIDA Y METRICA

In [27]:
#lr_schedule = tf.keras.optimizers.schedules.CosineDecay(learning_rate, decay_steps=100)
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[nmse])

#########################################################################################################<br>
#########################################################################################################<br>
#########################################################################################################

ENTRENAMIENTO DE LA RED<br>
Medir el tiempo de inicio del entrenamiento

In [28]:
start_time = time.time()
history = model.fit(X, Y, epochs=max_epoch, batch_size=batchsize, validation_split=validation_split)

Train on 21 samples, validate on 9 samples
Epoch 1/200
21/21 [==============================] - 13s 640ms/sample - loss: 11.7790 - nmse: 0.9683 - val_loss: 10.5006 - val_nmse: 0.8863
Epoch 2/200
21/21 [==============================] - 1s 48ms/sample - loss: 10.3604 - nmse: 0.8738 - val_loss: 9.2152 - val_nmse: 0.8527
Epoch 3/200
21/21 [==============================] - 1s 48ms/sample - loss: 9.0909 - nmse: 0.8572 - val_loss: 8.0566 - val_nmse: 0.8296
Epoch 4/200
21/21 [==============================] - 1s 48ms/sample - loss: 7.9473 - nmse: 0.8364 - val_loss: 7.0224 - val_nmse: 0.8204
Epoch 5/200
21/21 [==============================] - 1s 48ms/sample - loss: 6.9267 - nmse: 0.8214 - val_loss: 6.0998 - val_nmse: 0.8065
Epoch 6/200
21/21 [==============================] - 1s 48ms/sample - loss: 6.0171 - nmse: 0.8080 - val_loss: 5.2858 - val_nmse: 0.7998
Epoch 7/200
21/21 [==============================] - 1s 48ms/sample - loss: 5.2154 - nmse: 0.8059 - val_loss: 4.5700 - val_nmse: 0.7939


21/21 [==============================] - 1s 48ms/sample - loss: 0.3405 - nmse: 0.7560 - val_loss: 0.3176 - val_nmse: 0.7295
Epoch 61/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3353 - nmse: 0.7428 - val_loss: 0.3162 - val_nmse: 0.7282
Epoch 62/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3342 - nmse: 0.7461 - val_loss: 0.3142 - val_nmse: 0.7249
Epoch 63/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3317 - nmse: 0.7374 - val_loss: 0.3115 - val_nmse: 0.7193
Epoch 64/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3295 - nmse: 0.7256 - val_loss: 0.3096 - val_nmse: 0.7156
Epoch 65/200
21/21 [==============================] - 1s 51ms/sample - loss: 0.3272 - nmse: 0.7209 - val_loss: 0.3064 - val_nmse: 0.7080
Epoch 66/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3245 - nmse: 0.7169 - val_loss: 0.3048 - val_nmse: 0.7046
Epoch 67/200
21/21 [==============================] - 

Epoch 120/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3038 - nmse: 0.6607 - val_loss: 0.2847 - val_nmse: 0.6373
Epoch 121/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3015 - nmse: 0.6495 - val_loss: 0.2834 - val_nmse: 0.6339
Epoch 122/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3008 - nmse: 0.6556 - val_loss: 0.2819 - val_nmse: 0.6296
Epoch 123/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.2993 - nmse: 0.6324 - val_loss: 0.2842 - val_nmse: 0.6353
Epoch 124/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3030 - nmse: 0.6614 - val_loss: 0.3096 - val_nmse: 0.7030
Epoch 125/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3300 - nmse: 0.7448 - val_loss: 0.2974 - val_nmse: 0.6710
Epoch 126/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3115 - nmse: 0.6698 - val_loss: 0.2995 - val_nmse: 0.6777
Epoch 127/200
21/21 [=============

21/21 [==============================] - 1s 48ms/sample - loss: 0.2771 - nmse: 0.5459 - val_loss: 0.2667 - val_nmse: 0.5676
Epoch 180/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.2854 - nmse: 0.5883 - val_loss: 0.2887 - val_nmse: 0.6264
Epoch 181/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.3071 - nmse: 0.6408 - val_loss: 0.2664 - val_nmse: 0.5668
Epoch 182/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.2816 - nmse: 0.5796 - val_loss: 0.2734 - val_nmse: 0.5860
Epoch 183/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.2871 - nmse: 0.5644 - val_loss: 0.2677 - val_nmse: 0.5713
Epoch 184/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.2823 - nmse: 0.5709 - val_loss: 0.2638 - val_nmse: 0.5610
Epoch 185/200
21/21 [==============================] - 1s 48ms/sample - loss: 0.2797 - nmse: 0.5738 - val_loss: 0.2609 - val_nmse: 0.5533
Epoch 186/200
21/21 [===========================

Medir el tiempo de finalizaciOn del entrenamiento

In [29]:
end_time = time.time()

Calcular el tiempo total de entrenamiento

In [30]:
total_time = end_time - start_time
print(f'Tiempo total de entrenamiento: {total_time:.2f} segundos')

Tiempo total de entrenamiento: 238.45 segundos


#########################################################################################################<br>
#########################################################################################################<br>
#########################################################################################################

Visualizar el NMSE

In [ ]:
plt.plot(history.history['nmse'], label='NMSE (entrenamiento)')
plt.plot(history.history['val_nmse'], label='NMSE (validacion)')
plt.xlabel('Epoca')
plt.ylabel('NMSE')
plt.legend()
plt.show()